In [39]:
from frontmatter import Frontmatter

post = Frontmatter.read_file("./skills/generate-meta-prompt/SKILL.md")
metadata = post.get('attributes', None)
system_prompt = post.get('body', None)

In [40]:
metadata

{'name': 'generate-meta-prompt',
 'description': 'Generate effective prompts based on task descriptions. Use when you need to create structured prompts for AI models.'}

In [41]:
print(system_prompt)

# Meta Prompt Generator

You are an expert prompt engineer. Generate effective, structured prompts based on examples of input-output pairs.

## Instructions

Analyze the provided examples and create a prompt that will enable an AI to perform the same task consistently.

### Template Structure
```
# PERSONA
[Define the AI's role based on the task pattern]

# INSTRUCTION
- [Extract the core task from examples]
- [Identify key transformation rules]
- [Specify output format requirements]
- [Include edge case handling]

# EXAMPLES
[Include 1-2 representative examples from the provided set]
```

### Analysis Process
1. Examine all input-output pairs
2. Identify the consistent transformation pattern
3. Note formatting requirements
4. Extract any implicit rules or constraints
5. Generate clear, actionable instructions

Output only the generated prompt, no explanations.



In [69]:
import requests

base_url = "http://localhost:11434/api/generate"

def generate(prompt, model, system_prompt):
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "system": system_prompt,
        "options": dict(temperature=0.0, seed=55)
    }
    response = requests.post(base_url, json=data)
    if response.status_code == 200:
        response = response.json()
        return response["response"]
    else:
        print("Error:", response.status_code, response.text)

In [70]:
examples = """
INSTRUCTIONS
- correct the spelling

EXAMPLES:  

ex1:
INPUT: ผมไม่เจ้าใจกรมธรรน์เล่มนี้เลย
OUTPUT: ผมไม่เข้าใจกรมธรรม์เล่มนี้เลย

ex2:
INPUT: กรรมธรรม์ของผมมีความคุ้มครองอะไรบ้างครับ?
OUTPUT: กรมธรรม์ของผมมีความคุ้มครองอะไรบ้างครับ?

ex3:
INPUT: ช่วยค้นหาข้อมูลของ กธ. ให้พี่หน่อยจ้ะ
OUTPUT: ช่วยค้นหาข้อมูลของกรมธรรม์ให้พี่หน่อยจ้ะ
"""

In [71]:
spell_correction = generate(examples, model="llama3.2-vision:11b", system_prompt=system_prompt)
print(spell_correction)

# PERSONA
Thai Language Text Corrector

# INSTRUCTION
- Correct spelling errors in Thai text inputs
- Identify and replace incorrect spellings with their correct counterparts
- Preserve original text formatting and punctuation
- Handle edge cases where incorrect spellings are used in creative or colloquial expressions

# EXAMPLES

ex1:
INPUT: ผมไม่เจ้าใจกรมธรรน์เล่มนี้เลย
OUTPUT: ผมไม่เข้าใจกรมธรรม์เล่มนี้เลย

ex2:
INPUT: กรรมธรรม์ของผมมีความคุ้มครองอะไรบ้างครับ?
OUTPUT: กรมธรรม์ของผมมีความคุ้มครองอะไรบ้างครับ?

ex3:
INPUT: ช่วยค้นหาข้อมูลของ กธ. ให้พี่หน่อยจ้ะ
OUTPUT: ช่วยค้นหาข้อมูลของกรมธรรม์ให้พี่หน่อยจ้ะ


In [72]:
generate("INPUT: กรมทันไหนเหมาะสำหรับลูกค้าวัยทำงานบ้างคับ?", model="llama3.2-vision:11b", system_prompt=spell_correction)

'OUTPUT: กรมธรรมที่เหมาะสำหรับลูกค้าวัยทำงานบ้างคับ?'

In [73]:
generate("INPUT: กรรมทรรนที่ผมมีน่าจะได้เงินคืนตอนช่วงไหนคับ?", model="llama3.2-vision:11b", system_prompt=spell_correction)

'OUTPUT: กรมธรรม์ที่ผมมีน่าจะได้เงินคืนตอนช่วงไหนคับ?'

In [74]:
generate("INPUT: ดูให้หน่อยว่าตอนนี้ผมมีกี่กธ.แล้วตอนนี้", model="llama3.2-vision:11b", system_prompt=spell_correction)

'OUTPUT: ดูให้หน่อยว่าตอนนี้ผมมีกรมธรรม์เท่าไรแล้วตอนนี้'

In [75]:
generate("INPUT: ผมว่ามันไม่ควนที่จะคุ้คอรงแค่ค่าห้องนะ", model="llama3.2-vision:11b", system_prompt=spell_correction)

'OUTPUT: ผมว่ามันไม่ควรมีแค่ค่าห้องนะ'

In [82]:
generate("INPUT: ค่ารกษาพยาบาลต่อครั้งคือกีบาท?", model="llama3.2-vision:11b", system_prompt=spell_correction)

'OUTPUT: ค่าใช้จ่ายทางการแพทย์ต่อครั้งคือเงินกี่บาท?'